### Oracle Vector DB wrapped as a llama-index custom Vector Store

* ispired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* effects of **reranking**

In [1]:
import logging
import sys

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
import llama_index
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.postprocessor import FlagEmbeddingReranker

import oci
import ads

# only
import oracledb
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

from config import EMBED_MODEL
from config_private import COMPARTMENT_OCID, ENDPOINT, COHERE_API_KEY

In [2]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")
print(f"llama_index version: {llama_index.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649
llama_index version: 0.9.21


In [3]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=ads.auth.api_keys(oci_config),
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

#### Using the wrapper for the DB Vector Store

In [5]:
v_store = OracleVectorStore(verbose=False)

In [6]:
question = (
    "What is JSON Relational Duality in Oracle Database 23c? Explain with details"
)

In [7]:
# embed the query using OCI GenAI
query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=5)

#### Use our Vector Store DB

In [8]:
%%time

q_result = v_store.query(query_obj)

CPU times: user 22.1 ms, sys: 5.38 ms, total: 27.5 ms
Wall time: 769 ms


In [9]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Dod. id: {id}")
    print(f"Similarity: {-sim}")
    print(n.text)
    print("")

Dod. id: bfbaa15634643a2aac6e77b85a4f8a335eba697d741aaaa3646bdfb07fa3ba22
Similarity: 0.696
2 Application Development JSON JSON-Relational Duality JSON Relational Duality Views are fully updatable JSON views over relational data. Data is still stored in relational tables in a highly efficient normalized format but can be accessed by applications in the form of JSON documents. Duality views provide you with game-changing flexibility and simplicity by overcoming the historical challenges developers have faced when building applications using relational or document models. Related Resources View Documentation JSON Schema JSON Schema-based validation is allowed with the SQL condition IS JSON and with a PL/SQL utility function. A JSON schema is a JSON document that specifies allowed properties (field names) and the corresponding allowed data types, and whether they are optional or mandatory. By default, JSON data is schemaless, providing flexibility. However, you may want to ensure that you

#### Apply a reranker to the (question, text) pairs

In [10]:
# rerank = CohereRerank(api_key=COHERE_API_KEY, top_n=2)

In [11]:
rerank = FlagEmbeddingReranker(top_n=2, use_fp16=True)

In [12]:
rerank.model

'BAAI/bge-reranker-large'

#### Integrate in the bigger RAG picture

In [13]:
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

In [14]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [15]:
index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

In [16]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[rerank],
)

In [17]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print(response.response)
print("")

Question: What is JSON Relational Duality in Oracle Database 23c? Explain with details
JSON Relational Duality is a feature in Oracle Database 23c that allows developers to access and update data in both JSON documents and relational tables. This capability is particularly useful for those who want to work with data in a flexible schema format while still being able to enforce stricter data structures when necessary. Here's a more detailed explanation:

With JSON Relational Duality, you can store your data in highly efficient relational tables while simultaneously presenting a JSON document view of the same data to applications. This is particularly beneficial for scenarios where you might have applications that require JSON documents to interact with other services or systems, while the underlying data is best stored and managed in a relational manner.

Duality views, which are fully updatable JSON views over relational data, enable this functionality. These views present the data as 

#### Analyze response

In [18]:
for node in response.source_nodes:
    print(f"{node.text}\npag:{node.metadata['page_label']}\nscore:{node.score}")

1 Introduction Oracle Database 23c is the next long term support release of Oracle Database. Oracle Database 23c, code named “App Simple,” accelerates Oracle's mission to make it simple to develop and run all data-driven applications. It's the sum of all the features from the Oracle Database 21c innovation release plus over 300 new features and enhancements. Key focus areas include JSON, graph, microservices, and developer productivity. Note: For information about desupported features, see Oracle Database Changes, Desupports, and Deprecations. JSON Relational Duality Data can be transparently accessed and updated as either JSON documents or relational tables. Developers benefit from the strengths of both, which are simpler and more powerful than Object Relational Mapping (ORM). See JSON-Relational Duality . Operational Property Graphs in SQL Developers can now build real-time graph analysis applications against operational data directly in the Oracle Database, utilizing its industry le